In [1]:
#!conda install -n mldds -c anaconda joblib
import os
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

import multiprocessing
num_cores = multiprocessing.cpu_count()

print("Cores: ", num_cores)

import time
import keras
import tensorflow as tf
config = tf.ConfigProto( device_count = {'GPU': 0 , 'CPU': num_cores} )
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from MyTotoResearch import *

Cores:  12


/Users/walter/Software/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
Using TensorFlow backend.
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:923: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)
/Users/walter/Software/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/Users/walter/Software/anaconda3/lib/python3.6/site-packages/tensorflow/pyth

Done.


In [7]:
SVC?

In [2]:
def store_prediction(mrt, model, f):
    def getAllData(df):
        drop_cols = ['T', 'L','M','S','R','E','A','V' ,'J','U','K']
        X = df.drop(drop_cols, axis=1)
        return X

    test_data = mtr.get_test_data()
    X = mtr.modified_dataset(getAllData(test_data)) #
#    X = getAdjustedDataF(test_data,f)


    scaler = RobustScaler()
    scaler.fit(X)
    Z = scaler.transform(X)

#    Z = X
    predictions = model.predict(Z)
    dfResult= pd.DataFrame(predictions, columns=['N1', 'N2', 'N3', 'N4', 'N5','N6', 'N7'])
#    mtr.print_predictions(dfResult)

    global df_predictions
    global prev_r
    r = mtr.getAccuracyCount(np.array(dfResult)) ;
    if ( r > prev_r ):
#        df_predictions = []
        df_predictions.append(dfResult)


In [40]:
RidgeClassifier?

In [55]:
from keras.models import Input, Model
import keras
from keras.layers import Dense
import time
from keras.models import Sequential
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
import json as simplejson
from keras import regularizers
from sklearn import preprocessing
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR

from sklearn.linear_model import SGDRegressor, SGDClassifier, LogisticRegression, PassiveAggressiveClassifier, Perceptron, RidgeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Lasso, ElasticNet, Ridge, RidgeClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVC, SVR, LinearSVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}

def getAllData(df):
    drop_cols = ['T', 'D', 'N1','N2','N3','N4','N5','N6','N7','L','M','S','R','E','A','V' ,'J','U','K']
    X = df.drop(drop_cols, axis=1)
    return X

df_predictions = []

prev_r = 0

mtr = MyTotoResearch(algo_no=1)
lresult, df = mtr.load_totodata()

#Deep Neuro Network
for n in range(1,2):
    X = mtr.modified_dataset(getAllData(df)) #
    f = 1.0 #365/27.58
#    X = getAdjustedDataF(df,f)

    scaler = RobustScaler()
    scaler.fit(X)
    Z = scaler.transform(X)

#    Z = X

#    classifier = MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='tanh', learning_rate='adaptive', solver='sgd', verbose=0,  random_state=42,tol=0.000000001)
    clf = LogisticRegression(multi_class='multinomial', penalty='l1', fit_intercept=False, max_iter=5000, solver='saga') #, class_weight='balanced')
#    clf = RidgeClassifier(random_state=42, solver='cholesky', normalize=False, class_weight='balanced')
#    clf =  SVC(kernel='poly', coef0=0.06, probability=True, degree=2, random_state=42, tol=1e-05, shrinking=False)



#     param_grid = {
#                 'C': [1, 2] 
#                   ,'tol': [1e-6, 1e-05]
#                   , 'max_iter': [15000]
# #                  , 'random_state': [ 10, 20, 30, 40 ]
#                   , 'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']
#                  }
#     classifier = GridSearchCV(classifier, param_grid, cv=3)
# GridSearchCV(cv=None,
#        estimator=LogisticRegression(C=1.0, intercept_scaling=1, dual=False, fit_intercept=True,
#           penalty='l2', tol=0.0001),
#        fit_params={}, iid=True, loss_func=None, n_jobs=1,
#        param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]},
#        pre_dispatch='2*n_jobs', refit=True, score_func=None, verbose=0)

    model = MultiOutputClassifier(clf, n_jobs=7)
    model.fit(Z, mtr.getTargets()) 
    print(model)
    s = model.score(Z, mtr.getTargets())
    if(model.score(Z, mtr.getTargets()) == 1.0):
        print( str(f), ' ', str(s))
    store_prediction(mtr, model, f)
    start = time.clock()
    print(str(f), " Time taken: ", (time.clock() - start),  " ")

print("Done.")
# mtr = MyTotoResearch(algo_no=1)
# lresult, df = mtr.load_totodata()

# test_data = mtr.get_test_data()

for n in range(len(df_predictions)):
    print( mtr.getAccuracyCount(np.array(df_predictions[n])))
    mtr.print_predictions(df_predictions[n])


#69.81 => MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='relu', learning_rate='adaptive', solver='sgd', verbose=0,  random_state=42,tol=0.000000001)
#75.47 =>  MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='tanh', learning_rate='adaptive', solver='sgd', verbose=0,  random_state=42,tol=0.000000001)
#64.15 =>  MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='tanh', learning_rate='adaptive', solver='adam', verbose=0,  random_state=42,tol=0.000000001)
#62  MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='tanh', learning_rate='adaptive', solver='lbfgs', verbose=0,  random_state=42,tol=0.000000001)
#71.69 => MLPClassifier(hidden_layer_sizes=(500,500,500), max_iter=2000, alpha=0.001, activation='logistic', learning_rate='adaptive', solver='lbfgs', verbose=0,  random_state=42,tol=0.000000001)

#75.47 => SVC(kernel='poly', coef0=0.05, probability=True, degree=2, random_state=42, tol=1e-03)



#69.81 => SVC(random_state=42)



Loaded MyTotoResearch algo_no:  1
1521
MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
          intercept_scaling=1, max_iter=5000, multi_class='multinomial',
          n_jobs=None, penalty='l1', random_state=None, solver='saga',
          tol=0.0001, verbose=0, warm_start=False),
           n_jobs=7)
1.0  Time taken:  1.3000000000928935e-05  
Done.
67.9245283018868
20180514   [17 24 29 45 46 49  5]   [16.  6. 16. 26. 45. 27.]   [45]
20180517   [ 7 21 25 29 35 37 13]   [16.  6. 34. 26. 44. 27.]   []
20180521   [ 8 10 16 30 37 44 17]   [16.  6. 28. 26. 39. 27.]   [16]
20180524   [11 25 26 34 36 42 16]   [16.  6. 16. 26. 45. 27.]   [16 26]
20180528   [ 5  9 27 28 30 44  2]   [16.  6. 18. 18. 22. 38.]   []
20180531   [11 13 24 26 47 49 33]   [16.  6. 28. 26. 29. 43.]   [26]
20180604   [20 22 31 37 43 45 27]   [ 5.  6. 28. 26. 39. 38.]   []
20180607   [12 20 29 31 37 39 42]   [ 5.  6. 28. 26. 39. 41.]   [39]
20180611   [16 25 3

In [7]:
print(classifier.estimator)
# Nov 26
# 16 22 28 31 38 46 33

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)


In [35]:
LogisticRegression?

In [23]:
20180514   [17 24 29 45 46 49  5]   [ 8. 22. 16. 19. 39. 43.]   []
20180517   [ 7 21 25 29 35 37 13]   [16.  6. 34. 40. 35. 43.]   [35]
20180521   [ 8 10 16 30 37 44 17]   [ 5.  9. 16. 26. 24. 43.]   [16]
20180524   [11 25 26 34 36 42 16]   [ 5.  9. 16. 26. 45. 36.]   [16 26 36]
20180528   [ 5  9 27 28 30 44  2]   [ 5.  9. 16. 38. 39. 43.]   [5 9]
20180531   [11 13 24 26 47 49 33]   [20.  9. 23. 26. 34. 43.]   [26]
20180604   [20 22 31 37 43 45 27]   [ 5.  9. 16. 26. 22. 43.]   [22 43]
20180607   [12 20 29 31 37 39 42]   [ 8.  9. 16. 27. 22. 41.]   []
20180611   [16 25 30 37 44 49 34]   [ 5. 20. 25. 38. 44. 42.]   [25 44]
20180614   [ 4 29 31 35 42 48  1]   [18.  6. 18. 26. 32. 36.]   []
20180618   [11 15 22 23 26 43 25]   [18.  6. 16. 26. 22. 36.]   [22 26]
201806

75.47169811320755
20180514   [17 24 29 45 46 49  5]   [ 2. 35. 30. 17. 25. 27.]   [17]
20180517   [ 7 21 25 29 35 37 13]   [ 5. 33. 36. 16. 25. 31.]   [25]
20180521   [ 8 10 16 30 37 44 17]   [ 5. 27. 30. 18. 33. 38.]   [30]
20180524   [11 25 26 34 36 42 16]   [ 5. 26. 30. 18. 29. 45.]   [26]
20180528   [ 5  9 27 28 30 44  2]   [ 5.  7. 12. 18. 22. 38.]   [5]
20180531   [11 13 24 26 47 49 33]   [ 5. 35. 12. 38. 33. 41.]   [33]
20180604   [20 22 31 37 43 45 27]   [ 5.  7. 12. 18. 24. 38.]   []
20180607   [12 20 29 31 37 39 42]   [ 5. 35. 12. 38. 22. 40.]   [12]
20180611   [16 25 30 37 44 49 34]   [22. 27. 28. 38. 33. 38.]   []
20180614   [ 4 29 31 35 42 48  1]   [ 5. 36. 30. 23. 24. 31.]   [31]
20180618   [11 15 22 23 26 43 25]   [13. 36. 30. 17. 22. 36.]   [22]
20180621   [ 4  6 15 24 30 35 46]   [10. 20. 23. 28. 32. 42.]   []
20180625   [ 2  5 25 38 44 48  9]   [15. 18. 20. 27. 41. 40.]   []
20180628   [ 2  7 22 27 40 47 48]   [12. 15. 27. 35. 41. 43.]   [27]
20180702   [12 13 26 33 35 38 23]   [ 8.  7. 19. 35. 41. 43.]   [35]
20180705   [ 8 11 28 30 32 34 39]   [ 9. 18. 17. 28. 37. 42.]   [28]
20180709   [ 6 23 31 38 39 43 33]   [12. 23. 26. 28. 36. 40.]   [23]
20180712   [ 4 15 25 32 40 41 10]   [ 5. 23. 35. 29. 32. 45.]   [32]
20180716   [ 4  8 19 24 32 47 22]   [ 8. 19. 27. 24. 31. 38.]   [ 8 19 24]
20180719   [13 14 23 35 37 46 45]   [10. 17. 27. 32. 40. 41.]   []
20180723   [ 2 23 26 28 39 40 12]   [23. 23. 33. 28. 31. 43.]   [23 28]
20180726   [ 1  9 13 17 28 40 37]   [11. 28. 30. 32. 34. 39.]   [28]
20180730   [ 8 10 19 20 41 43  7]   [ 8. 23. 26. 24. 35. 38.]   [8]
20180802   [ 1 10 15 27 41 46 35]   [ 3.  9. 25. 29. 40. 48.]   []
20180806   [ 7 18 20 27 36 40 15]   [ 6. 15. 24. 30. 35. 45.]   [15]
20180809   [13 16 20 23 39 42 28]   [11. 22. 23. 27. 35. 43.]   [23]
20180813   [ 1  3  6 16 22 36 17]   [ 4. 18. 31. 35. 42. 44.]   []
20180816   [22 23 25 32 33 36 20]   [ 7. 18. 34. 31. 34. 44.]   []
20180820   [ 9 10 25 38 40 42  2]   [ 9. 18. 26. 27. 39. 44.]   [9]
20180823   [ 2  3 23 30 39 41 19]   [ 8. 18. 23. 37. 43. 44.]   [23]
20180827   [ 5  6 16 24 26 29 38]   [ 8. 17. 23. 27. 41. 44.]   []
20180830   [ 3  9 27 29 31 40 46]   [ 8. 18. 27. 32. 41. 44.]   [27]
20180903   [ 4  5 13 18 39 40  3]   [10. 18. 22. 27. 36. 42.]   [18]
20180906   [ 2 15 17 20 23 30 45]   [ 7. 18. 23. 27. 41. 44.]   [23]
20180910   [ 2  6  9 15 40 43 18]   [ 7. 18. 19. 27. 41. 44.]   [18]
20180913   [ 6 16 17 40 44 48 34]   [11. 18. 17. 27. 41. 44.]   [17 44]
20180917   [16 21 22 24 25 27  1]   [ 7. 18. 17. 27. 41. 44.]   [27]
20180920   [ 5 12 18 30 32 38 22]   [11. 18. 22. 27. 41. 44.]   [18 22]
20180924   [ 6  8 17 24 29 47 34]   [ 7. 18. 19. 31. 38. 45.]   []
20180927   [ 2 25 29 33 42 45 20]   [11. 18. 23. 31. 41. 44.]   []
20181001   [11 15 23 24 32 40 43]   [11. 18. 17. 31. 41. 44.]   [11]
20181004   [ 5 12 23 32 37 42 43]   [ 7. 18. 22. 27. 41. 44.]   []
20181008   [17 18 23 39 43 49  2]   [ 4. 11. 22. 30. 30. 45.]   []
20181011   [ 1 16 18 24 29 46 35]   [ 9. 18. 23. 32. 41. 43.]   [18]
20181015   [ 1  4 24 32 35 48 20]   [ 7. 18. 17. 31. 35. 45.]   [35]
20181018   [ 5 14 17 31 46 48 47]   [ 5. 13. 17. 31. 40. 45.]   [ 5 17 31]
20181022   [ 5 22 24 40 43 48  2]   [ 5. 15. 25. 31. 38. 42.]   [5]
20181025   [ 7  8 13 15 35 48 30]   [14. 17. 22. 34. 38. 42.]   []
20181029   [ 2  6 10 20 28 31 30]   [ 6. 15. 19. 31. 40. 44.]   [ 6 31]
20181101   [ 6 27 28 41 44 48 15]   [ 6. 15. 23. 27. 41. 44.]   [ 6 15 27 41 44]
20181101   [ 6 27 28 41 44 48 15]   [ 8. 15. 29. 27. 34. 41.]   [15 27 41]

20181105   [ 3  8 14 28 43 49 26]   [11. 18. 23. 31. 41. 44.]   []
20181108   [ 8 13 16 26 28 38 46]   [ 7. 17. 23. 31. 38. 45.]   [38]
20181112   [ 4 12 21 34 41 47 33]   [ 8. 12. 22. 31. 41. 44.]   [12 41]
20181115  Predicted:  [10. 18. 22. 27. 40. 44.]  
20181119  Predicted:  [ 9. 18. 19. 27. 35. 45.]  
20181122  Predicted:  [ 9. 18. 22. 27. 35. 41.]  
20181126  Predicted:  [ 6. 18. 23. 27. 38. 44.]  
20181129  Predicted:  [ 7. 18. 23. 30. 35. 44.]  
20181203  Predicted:  [ 7. 18. 17. 31. 41. 45.]  
20181206  Predicted:  [ 7. 18. 17. 27. 35. 44.]  
20181210  Predicted:  [ 6. 18. 17. 31. 41. 45.]  
20181213  Predicted:  [ 7. 18. 17. 31. 41. 44.]  
20181217  Predicted:  [ 7. 18. 19. 27. 35. 44.]  
20181220  Predicted:  [ 7. 18. 23. 31. 41. 44.]  
20181224  Predicted:  [ 7. 18. 22. 27. 41. 44.]  
20181227  Predicted:  [ 8. 18. 23. 27. 41. 44.]  
20181231  Predicted:  [ 9. 18. 23. 27. 41. 44.]  

In [19]:
#Keep track of all results
#df_predictions = []

#print(df_predictions)
#mtr = MyTotoResearch(algo_no=1)
def getAllData(df):
    drop_cols = ['T', 'L','M','S','R','E','A','V' ,'J','U','K']
    X = df.drop(drop_cols, axis=1)
    return X

lresult, df = mtr.load_totodata()

test_data = mtr.get_test_data()
X = mtr.modified_dataset(getAllData(test_data)) #

print(len(df_predictions))
for n in range(len(df_predictions)):
    print( mtr.getAccuracyCount(np.array(df_predictions[n])))
    mtr.print_predictions(df_predictions[n])



1521
1
62.264150943396224
20180514   [17 24 29 45 46 49  5]   [16. 21. 22. 24. 25. 27.]   [24]
20180517   [ 7 21 25 29 35 37 13]   [16. 17. 34. 40. 44. 48.]   []
20180521   [ 8 10 16 30 37 44 17]   [ 6.  9. 15. 18. 40. 43.]   []
20180524   [11 25 26 34 36 42 16]   [15. 17. 20. 23. 30. 45.]   []
20180528   [ 5  9 27 28 30 44  2]   [ 4.  5. 13. 18. 39. 40.]   [5]
20180531   [11 13 24 26 47 49 33]   [ 9. 27. 29. 31. 40. 46.]   []
20180604   [20 22 31 37 43 45 27]   [ 6. 16. 24. 26. 29. 38.]   []
20180607   [12 20 29 31 37 39 42]   [ 3. 19. 23. 30. 39. 41.]   [39]
20180611   [16 25 30 37 44 49 34]   [ 9. 10. 25. 38. 40. 42.]   [25]
20180614   [ 4 29 31 35 42 48  1]   [22. 23. 25. 32. 33. 36.]   []
20180618   [11 15 22 23 26 43 25]   [ 3.  6. 16. 17. 22. 36.]   [22]
20180621   [ 4  6 15 24 30 35 46]   [16. 20. 23. 28. 39. 42.]   []
20180625   [ 2  5 25 38 44 48  9]   [15. 18. 20. 27. 36. 40.]   []
20180628   [ 2  7 22 27 40 47 48]   [10. 15. 27. 35. 41. 46.]   [27]
20180702   [12 13 26 33 3

In [ ]:
dOld = dfResult
#2 9 14 36 46 48 5